Kvasir V2 dataset trained on all classes with resnet50 without augmentation.

### Loading images
https://www.tensorflow.org/tutorials/load_data/images

## Training

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

import numpy as np
import os
import pathlib
import matplotlib.pyplot as plt

# Some stuff to make utils-function work
import sys
sys.path.append('/home/henrik/master_thesis/code/utils')
from data_prep import create_dataset, print_class_info, show_image
%load_ext autoreload
%autoreload 2

# Jupyter-specific
%matplotlib inline

Some parameters

In [2]:
data_dir = pathlib.Path('/mnt/sdb/kvasir-dataset-v2/')

config = {
    "data_dir": data_dir,
    "cache_dir": "./cache",
    "MODEL": 'resnet50',
    "DS_INFO": 'complete',
    "resample": False,
    "neg_class": ['polyp'],
    "NUM_EPOCHS": 100,
    "BATCH_SIZE": 64,
    "IMG_SIZE": (32, 32, 3),
    "outcast": None,
    "verbosity": 1
    }

model_name = '{}x{}x{}_{}_{}'.format(config["NUM_EPOCHS"], config["BATCH_SIZE"], 
                                     config["IMG_SIZE"][1], config["DS_INFO"], config["MODEL"])

learning_rate = 0.01
fine_tune_at = 130
fine_tune_epochs = 50

Create training, testing and validation dataset from utils/data_prep.py

In [3]:
train_ds, test_ds, val_ds, params = create_dataset(config)

Directories:  ['esophagitis' 'normal-pylorus' 'ulcerative-colitis' 'normal-cecum'
 'normal-z-line' 'polyps' 'dyed-resection-margins' 'dyed-lifted-polyps']

esophagitis       : 1000
normal-pylorus    : 1000
ulcerative-colitis: 1000
normal-cecum      : 1000
normal-z-line     : 1000
polyps            : 1000
dyed-resection-margins: 1000
dyed-lifted-polyps: 1000

Total number of images: 8000, in 8 classes
Dataset.list_files:  /mnt/sdb/kvasir-dataset-v2/*/*.*g 

[6 1 4 5 1 5 7 0 3 5]
[4 4 3 3 7 2 3 3 6 2]
[3 6 1 4 1 4 0 1 3 7]
[7 2 7 1 0 5 4 3 5 0]
[7 2 3 3 0 5 5 4 7 4]
[0 6 2 6 2 3 1 3 7 6]
[1 0 4 5 3 3 6 7 4 0]
[4 3 1 5 4 4 4 7 7 1]
[1 0 4 4 5 5 0 7 7 6]
[5 3 7 2 3 2 4 6 3 6]

Full dataset sample size:         8000
Train dataset sample size:        5600
Test dataset sample size:         1200
Validation dataset sample size:   1200


### Resnet50 transfer learning
see: https://adventuresinmachinelearning.com/transfer-learning-tensorflow-2/  
fine-tuning: https://www.tensorflow.org/tutorials/images/transfer_learning

In [4]:
from tensorflow.python.keras.applications import ResNet50
from tensorflow.python.keras.models import Sequential
from tensorflow.keras import layers

Create and compile the model

In [5]:
for image_batch, label_batch in train_ds.take(1):
    pass

image_batch.shape

TensorShape([64, 32, 32, 3])

In [6]:
base_model = tf.keras.applications.ResNet50(
                weights='imagenet', 
                include_top=False, 
                input_shape=config["IMG_SIZE"])

feature_batch = base_model(image_batch)

In [7]:
base_model.trainable = True

global_average_layer = layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)


prediction_layer = layers.Dense(params["NUM_CLASSES"], activation='softmax')
prediction_batch = prediction_layer(feature_batch_average)

resnet50_model = tf.keras.Sequential([
        base_model,
        global_average_layer,
        prediction_layer])

In [8]:
if config["verbosity"] > 0:
    resnet50_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 1, 1, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 8)                 16392     
Total params: 23,604,104
Trainable params: 23,550,984
Non-trainable params: 53,120
_________________________________________________________________


In [9]:
opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)

resnet50_model.compile(
        optimizer=opt,
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy'])

callbacks = [tf.keras.callbacks.TensorBoard(
                log_dir='./logs/resnet50_model', 
                update_freq='batch')]

Train the model

In [ ]:
history = resnet50_model.fit(
        train_ds,
        steps_per_epoch = params["train_size"] // config["BATCH_SIZE"],
        epochs = config["NUM_EPOCHS"],
        validation_data = test_ds,
        validation_steps = params["test_size"] // config["BATCH_SIZE"],
        validation_freq = 1,
        callbacks = callbacks)

Train for 87 steps, validate for 18 steps
Epoch 1/100
87/87 [==============================] - 12s 138ms/step - loss: 4.3013 - accuracy: 0.1620 - val_loss: 27157515.1111 - val_accuracy: 0.1363
Epoch 2/100
87/87 [==============================] - 4s 43ms/step - loss: 3.2259 - accuracy: 0.2051 - val_loss: 2.0877 - val_accuracy: 0.1267
Epoch 3/100
87/87 [==============================] - 4s 43ms/step - loss: 2.2194 - accuracy: 0.2920 - val_loss: 2.8408 - val_accuracy: 0.1293
Epoch 4/100
87/87 [==============================] - 4s 42ms/step - loss: 4.0288 - accuracy: 0.2821 - val_loss: 341564.5260 - val_accuracy: 0.1285
Epoch 5/100
87/87 [==============================] - 4s 42ms/step - loss: 2.0620 - accuracy: 0.2940 - val_loss: 4.4817 - val_accuracy: 0.1658
Epoch 6/100
87/87 [==============================] - 4s 42ms/step - loss: 1.7168 - accuracy: 0.3628 - val_loss: 5.7315 - val_accuracy: 0.1181
Epoch 7/100
81/87 [==========================>...] - ETA: 0s - loss: 1.3412 - accuracy: 0.46

`Keras`
Save/load the model

In [ ]:
# resnet50_model.save('models/{}.h5'.format(model_name))
# resnet50_model = tf.keras.models.load_model('models/{}.h5'.format(model_name))

## Evaluate the model

In [ ]:
resnet50_evaluate = resnet50_model.evaluate(val_ds, verbose=2, steps=params["val_size"] // config["BATCH_SIZE"])

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(config["NUM_EPOCHS"])

In [ ]:
plt.figure(figsize=(14, 6))

# Subplot 1
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
# plt.ylim([0.5, 1])
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')

# Subplot 2
plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
# plt.ylim([0.5, 1])
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and Validation Loss')
plt.show()

### Fine tune the model

In [ ]:
base_model.trainable = True

In [ ]:
# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(base_model.layers))

# Fine-tune from this layer onwards
fine_tune_at = fine_tune_at

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable =  False

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=learning_rate/10)

resnet50_model.compile(
        optimizer=opt,
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy'])

In [ ]:
len(resnet50_model.trainable_variables)

In [ ]:
fine_tune_epochs = fine_tune_epochs
total_epochs = config["NUM_EPOCHS"] + fine_tune_epochs

history_fine = resnet50_model.fit(
        train_ds,
        steps_per_epoch = params["train_size"] // config["BATCH_SIZE"],
        epochs = total_epochs,
        initial_epoch = history.epoch[-1],
        validation_data = test_ds,
        validation_steps = params["test_size"] // config["BATCH_SIZE"],
        validation_freq = 1,
        callbacks = callbacks)

In [ ]:
# For fine-tuning
acc += history_fine.history['accuracy']
val_acc += history_fine.history['val_accuracy']

loss += history_fine.history['loss']
val_loss += history_fine.history['val_loss']

In [ ]:
plt.figure(figsize=(14, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
# plt.ylim([0.2, 1])
plt.plot([config["NUM_EPOCHS"]-1,config["NUM_EPOCHS"]-1],
          plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.ylim([0, 3.0])
plt.plot([config["NUM_EPOCHS"]-1,config["NUM_EPOCHS"]-1],
         plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

`Tensorboard`

## Predict new samples

In [ ]:
# Fetch one batch
images, labels = next(iter(val_ds))

# Convert from tensor to numpy array
images = images.numpy()
labels = labels.numpy()

In [ ]:
# Get a random image and label
rand = np.random.randint(0, config["BATCH_SIZE"])
image = images[rand]
label = labels[rand]

# Predict one image
predictions = resnet50_model.predict(np.expand_dims(image, axis=0))[0]

for i, pred in enumerate(predictions):
    print("{:0.4f} {}".format(pred,params["class_names"][i]))

print ("\nLabel:", params["class_names"][label])
print ("Predicton:", params["class_names"][np.argmax(predictions)])

plt.figure(frameon=False, facecolor='white')
plt.imshow(image)
plt.axis('off');

In [ ]:
# Predict one batch
predictions = resnet50_model.predict(images)

print ('{:3}  {:<25} {:25}'.format('idx', 'label', 'pred'))
print ('---  {} {}'.format(25*'-', 25*'-'), end='')

for i, pred in enumerate(predictions):
    label = params["class_names"][labels[i]]
    prediction = params["class_names"][np.argmax(pred)]
    print ('\n{:3}  {:25} {:25}'.format(i, label, prediction), end='')
    if (label != prediction): print ("  Wrong", end='')